# CS 533 Project: TVCA - Traffic volumes correlation with bike traffic
by Abishai Joy and Ravishankar

## Purpose and Context
The purpose of this notebook is to find the correlation between the toal traffic volume on a street and the bike traffic on a street

## Setup